In [1]:
import heapq  # Para usar uma fila de prioridade (min-heap)

In [25]:
def dijkstra(grafo, origem):
    # Calcula a menor distância de um ponto de origem para todos os outros
    dist = {nodo: float('inf') for nodo in grafo}
    dist[origem] = 0
    fila = [(0, origem)]  # (distância, nodo)

    while fila:
        dist_atual, nodo_atual = heapq.heappop(fila)

        if dist_atual > dist[nodo_atual]:
            continue

        for vizinho, peso in grafo[nodo_atual]:
            nova_dist = dist_atual + peso

            if nova_dist < dist[vizinho]:
                dist[vizinho] = nova_dist
                heapq.heappush(fila, (nova_dist, vizinho))

    return dist

In [9]:
from datetime import datetime, timedelta

# D? -> destino
encomendas = {
    'D1': [
        {
            'id_encomenda': 'E011',
            'centro_distribuicao': 'CD4',
            'prazo_entrega': datetime.now() + timedelta(days=3),
            'peso': 5000
        },
        {
            'id_encomenda': 'E012',
            'centro_distribuicao': 'CD1',
            'prazo_entrega': datetime.now() + timedelta(days=5),
            'peso': 5000
        }
    ],
    'D2': [
        {
            'id_encomenda': 'E021',
            'centro_distribuicao': 'CD3',
            'prazo_entrega': datetime.now() + timedelta(days=2),
            'peso': 5000
        }
    ],
    'D3': [
        {
            'id_encomenda': 'E031',
            'centro_distribuicao': 'CD5',
            'prazo_entrega': datetime.now() + timedelta(days=4),
            'peso': 5000
        }
    ]
}

In [27]:
# CD? -> centro de distribuição
caminhoes = {
    "CD1": [
        {
            "id_caminhao" : "C011",
            "peso_maximo" : 5000,
            "limite_operacional": 8
        },
        {
            "id_caminhao" : "C012",
            "peso_maximo" : 6000,
            "limite_operacional": 5
        },
    ],
    "CD2": [
        {
            "id_caminhao" : "C021",
            "peso_maximo" : 8000,
            "limite_operacional": 8
        },
    ],
    "CD3": [
        {
            "id_caminhao" : "C031",
            "peso_maximo" : 10000,
            "limite_operacional": 10
        },
    ],
    "CD4": [
        {
            "id_caminhao" : "C041",
            "peso_maximo" : 10000,
            "limite_operacional": 10
        },
    ],
    "CD5": [
        {
            "id_caminhao" : "C051",
            "peso_maximo" : 16000,
            "limite_operacional": 8
        },
    ]
}

In [40]:
# Armazena o resultado de cada encomenda
entregas = {}

In [10]:
# (vizinho, distancia em horas)
grafo = {
    'CD1': [('D1', 4), ('D2', 10), ('D3', 2)],
    'CD2': [('D1', 10), ('D2', 3), ('D3', 7)],
    'CD3': [('D1', 5), ('D2', 8), ('D3', 10)],
    'CD4': [('D1', 2), ('D2', 8), ('D3', 8)],
    'CD5': [('D1', 6), ('D2', 6), ('D3', 6)],
    'D1': [], 'D2': [], 'D3': []  # Destinos não têm arestas saindo deles
}

In [23]:
# Calcula as menores distâncias de cada CD para todos os destinos
resultados = {}
for cd in ['CD1', 'CD2', 'CD3', 'CD4', 'CD5']:
    resultados[cd] = dijkstra(grafo, cd)

In [33]:
# Ordena o cds do mais proximo ao mais distante em relação ao destino
distancias_destino = {}

for destino in encomendas:
    cds_to_destino = []

    for cd in resultados:
        cds_to_destino.append((cd, resultados[cd][destino]))

    distancias_destino[destino] = sorted(cds_to_destino, key=lambda x: x[1])

print(distancias_destino)

{'D1': [('CD4', 2), ('CD1', 4), ('CD3', 5), ('CD5', 6), ('CD2', 10)], 'D2': [('CD2', 3), ('CD5', 6), ('CD3', 8), ('CD4', 8), ('CD1', 10)], 'D3': [('CD1', 2), ('CD5', 6), ('CD2', 7), ('CD4', 8), ('CD3', 10)]}


In [70]:
# Armazena o resultado de cada encomenda
entregas = {}

caminhoes_ocupados = [];

for destino in encomendas:
    for encomenda in encomendas[destino]:

        # Verifica se algum dos caminhoes daquele CD tem disponibilidade para entrega, se não, passa para o segundo CD mais proximo
        for index_cd_mais_proximo in range(len(distancias_destino[destino])):
            if(encomenda['id_encomenda'] in entregas): break

            for caminhao in caminhoes[distancias_destino[destino][index_cd_mais_proximo][0]]:

                if(caminhao['peso_maximo'] > encomenda['peso'] and caminhao['id_caminhao'] not in caminhoes_ocupados): # Adicionar a validação de prazo de entrega

                    entregas[encomenda['id_encomenda']] = {
                        'id_caminhao': caminhao['id_caminhao'],
                        'distancia': distancias_destino[destino][0][1],
                        'prazo_entrega': encomenda['prazo_entrega'],
                        'destino': destino,
                        'peso': encomenda['peso']
                    }
                    caminhoes_ocupados.append(caminhao['id_caminhao'])
                    break

print(entregas)

{'E011': {'id_caminhao': 'C041', 'distancia': 2, 'prazo_entrega': datetime.datetime(2024, 11, 12, 18, 22, 58, 263059), 'destino': 'D1', 'peso': 5000}, 'E012': {'id_caminhao': 'C012', 'distancia': 2, 'prazo_entrega': datetime.datetime(2024, 11, 14, 18, 22, 58, 263093), 'destino': 'D1', 'peso': 5000}, 'E021': {'id_caminhao': 'C021', 'distancia': 3, 'prazo_entrega': datetime.datetime(2024, 11, 11, 18, 22, 58, 263096), 'destino': 'D2', 'peso': 5000}, 'E031': {'id_caminhao': 'C051', 'distancia': 2, 'prazo_entrega': datetime.datetime(2024, 11, 13, 18, 22, 58, 263098), 'destino': 'D3', 'peso': 5000}}


In [69]:
for encomenda in entregas:
    print(f"A encomenda {encomenda} de {entregas[encomenda]['peso']} foi entregue pelo caminho {entregas[encomenda]['id_caminhao']} em {entregas[encomenda]['distancia']} horas.")

A encomenda E011 de 5000 foi entregue pelo caminho C041 em 2 horas.
A encomenda E012 de 5000 foi entregue pelo caminho C012 em 2 horas.
A encomenda E021 de 5000 foi entregue pelo caminho C021 em 3 horas.
A encomenda E031 de 5000 foi entregue pelo caminho C051 em 2 horas.
